In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用Vertex AI在电子商务数据上进行库存预测

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/inventory-prediction/inventory_prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/inventory-prediction/inventory_prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/workbench/inventory-prediction/inventory_prediction.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI工作台中打开
    </a>
  </td>                                                                                               
</table>

注意：此笔记本已在以下环境中进行了测试：

- Python版本=3.9

## 概述

本笔记本探讨了如何基于电子商务数据集构建一个用于库存预测的机器学习模型。该笔记本包括使用Vertex AI SDK在Vertex AI上部署模型的步骤，并使用What-If Tool分析部署的模型。了解更多关于[What-If Tool](https://pair-code.github.io/what-if-tool/)的信息。

了解更多关于[Vertex AI Workbench](https://cloud.google.com/vertex-ai/docs/workbench/introduction)和[Vertex AI Training](https://cloud.google.com/vertex-ai/docs/training/custom-training)。

注意：What-IF工具小部件在Colab和Vertex AI Workbench的托管实例上经过测试。可能无法在用户管理的实例上正常工作。

### 目标

本教程向您展示如何进行探索性数据分析，预处理数据，训练模型，评估模型，部署模型，并配置 What-If 工具。

本教程使用以下 Google Cloud ML 服务和资源:

- Vertex AI 模型
- Vertex AI 端点
- Vertex 可解释 AI
- Google Cloud 存储
- BigQuery

执行的步骤包括:

* 使用 "笔记本中的 BigQuery" 集成从 BigQuery 加载数据集。
* 分析数据集。
* 预处理数据集中的特征。
* 构建一个随机森林分类器模型，预测产品在接下来的60天内是否会卖出。
* 评估模型。
* 使用 Vertex AI 部署模型。
* 配置并使用 What-If 工具进行测试。

### 数据集

这个笔记本中使用的数据集包含自2018年以来电子商务商店的库存数据。这个数据集作为一个BigQuery表公开可用，表名为`looker-private-demo.ecomm.inventory_items`，可以通过在BigQuery中固定`looker-private-demo`项目来访问。该表包含与电子商务库存项目相关的各种字段，如`id`、`product_id`、`cost`、商品到达商店的时间、商品出售的时间等。这个笔记本使用以下字段，假设它们的用途如下所述：

- `id`：库存项目的ID
- `product_id`：产品的ID
- `created_at`：商品到达库存/商店时的时间
- `sold_at`：商品销售的时间（*如果尚未售出则为空*）
- `cost`：商品销售时的成本
- `product_category`：产品的类别
- `product_brand`：产品的品牌（稍后会被删除，因为值太多）
- `product_retail_price`：产品的价格
- `product_department`：产品所属的部门
- `product_distribution_center_id`：产品销售自哪个分销中心（区域的近似值）

数据集已经进行编码以隐藏任何私人信息。例如，分销中心已经被分配了从1到10的ID编号。

### 费用

此教程使用以下 Google Cloud 的计费组件：

- Vertex AI
- BigQuery
- Cloud Storage

了解关于 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing), [BigQuery 价格](https://cloud.google.com/bigquery/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing) 的信息，并使用 [价格计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成成本估算。

## 安装

安装以下所需的软件包以执行这个笔记本。

In [ ]:
! pip3 install --quiet --upgrade google-cloud-aiplatform \
                                    google-cloud-storage \
                                    seaborn \
                                    pandas \
                                    fsspec \
                                    witwidget \
                                    pyarrow \
                                    db-dtypes \
                                    gcsfs

! pip3 install scikit-learn==1.2 protobuf==3.20.1

只有Colab：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 开始之前

### 设置您的 Google 云项目

**无论您使用的是哪个笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个 Google 云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300 的免费信用额度，可用于计算/存储成本。

2. [确保为您的项目启用了计费功能](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

#### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
- 运行 `gcloud config list`
- 运行 `gcloud projects list`
- 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。
了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 认证您的 Google Cloud 账户

根据您的 Jupyter 环境，您可能需要手动进行身份验证。请按照以下相关说明进行操作。

**1. Vertex AI Workbench** 
- 如果您已经进行了身份验证，则无需进行任何操作。

**2. 本地 JupyterLab 实例，请取消注释并运行。**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账号或其他
- 查看所有身份验证选项请点击这里：[Google Cloud Platform Jupyter Notebook 身份验证指南](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/notebook_authentication_guide.ipynb)

创建一个云存储桶

创建一个存储桶，用于存储诸如数据集之类的中间产物。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的桶尚不存在时才能运行以下单元格来创建您的云存储桶。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

###导入库

In [ ]:
import os
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
from google.cloud import aiplatform, storage
from google.cloud.bigquery import Client
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from witwidget.notebook.visualization import WitConfigBuilder, WitWidget

### 初始化 Python 的 Vertex AI SDK

为您的项目初始化 Python 的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

从BigQuery加载所需数据

以下单元格通过Vertex AI的“Notebooks中的BigQuery”集成与同一项目中的BigQuery数据集成。它可以运行一个类似于在BigQuery控制台中运行的SQL查询。

*注意:* 此功能仅在运行在Vertex AI Workbench托管笔记本实例上的笔记本中可用。

#@bigquery
SELECT 
    id,
    product_id, 
    created_at,
    sold_at,
    cost,
    product_category,
    product_brand,
    product_retail_price,
    product_department,
    product_distribution_center_id
FROM 
looker-private-demo.ecomm.inventory_items

执行上述单元格后，单击**查询并加载为数据框**按钮会添加以下Python单元格，将查询的数据加载到Pandas数据框中。

In [ ]:
# The following two lines are only necessary to run once.
# Comment out otherwise for speed-up.
client = Client(project=PROJECT_ID)

query = """SELECT 
    id,
    product_id, 
    created_at,
    sold_at,
    cost,
    product_category,
    product_brand,
    product_retail_price,
    product_department,
    product_distribution_center_id
FROM 
looker-private-demo.ecomm.inventory_items"""
job = client.query(query)
df = job.to_dataframe()

## 探索并清洗数据集

检查数据集的前五行。

In [ ]:
df.head(5)

检查数据集中的字段及其数据类型和空值数量。

In [ ]:
df.info()

除了`sold_at`日期时间字段外，数据集中没有任何包含空值的字段。由于您正在处理库存物品数据，因此有些物品可能尚未售出，因此会出现空值。

### 清理日期时间字段
接下来，将日期字段转换为适当的日期格式，以便在下一步中处理。

In [ ]:
# convert to proper date columns
df["created_at"] = pd.to_datetime(df["created_at"], format="%Y-%m-%d")
df["sold_at"] = pd.to_datetime(df["sold_at"].dt.strftime("%Y-%m-%d"))

检查日期范围。

In [ ]:
# check the date ranges
print("Min-sold_at : ", df["sold_at"].min())
print("Max-sold_at : ", df["sold_at"].max())

print("Min-created_at : ", df["created_at"].min())
print("Max-created_at : ", df["created_at"].max())

### 提取有用的特征

从日期字段`created_at`中提取月份。

In [ ]:
# calculate the month when the item has arrived
df["arrival_month"] = df["created_at"].dt.month

计算产品在库存中直至卖出的平均天数。

In [ ]:
# calculate the number of days the item hasn't been sold.
df["shelf_days"] = (df["sold_at"] - df["created_at"]).dt.days

计算适用于产品的折扣百分比。

In [ ]:
# calculate the discount offered
df["discount_perc"] = (df["product_retail_price"] - df["cost"]) / df[
    "product_retail_price"
]

检查分类字段
检查数据中的独特产品及其品牌。

In [ ]:
# check total unique items
df["product_id"].unique().shape, df["product_brand"].unique().shape

字段`product_id`和`product_brand`似乎有很多唯一值。为了预测的目的，使用`product_id`作为主键，删除`product_brand`，因为它具有太多的值/级别。

分离需要的数字和分类字段以分析数据集。

In [ ]:
categ_cols = [
    "product_category",
    "product_department",
    "product_distribution_center_id",
    "arrival_month",
]
num_cols = ["cost", "product_retail_price", "discount_perc", "shelf_days"]

检查每个分类字段的各个类别的计数。

In [ ]:
for i in categ_cols:
    print(i, " - ", df[i].unique().shape[0])

检查数字字段的分布。

In [ ]:
df[num_cols].describe().T

将数据分布可视化

为分类字段生成条形图，并为数值字段生成直方图和箱线图，以检查数据集中它们的分布情况。

In [ ]:
for i in categ_cols:
    df[i].value_counts(normalize=True).plot(kind="bar")
    plt.title(i)
    plt.show()

for i in num_cols:
    _, ax = plt.subplots(1, 2, figsize=(10, 4))
    df[i].plot(kind="box", ax=ax[0])
    df[i].plot(kind="hist", ax=ax[1])
    ax[0].set_title(i + "-Boxplot")
    ax[1].set_title(i + "-Histogram")
    plt.show()

大多数字段，如折扣、部门、配送中心-标识符，具有合理的分布。对于字段“产品类别”，有一些类别至少不构成数据集的2%。虽然在一些数值字段中存在异常值，但它们被豁免不被移除，因为可能有一些价格昂贵或属于不经常看到许多销售的特定类别的产品。

## 特征预处理

接下来，基于数据中合适的分类字段对数据进行聚合，并获取产品售出所需的平均天数。对于给定的“产品标识符”，在此数据集中可能有多个项目标识符，并且您希望在产品级别上预测该特定产品是否将在接下来的几个月内销售。您正在根据数据集中存在的每个产品配置对数据进行聚合，例如价格、成本、类别以及在哪个中心出售。通过这种方式，模型可以预测具有某些属性的产品是否将在接下来的几个月内销售。

### 生成聚合特性

对于产品售出的天数，找到“上架天数”字段的平均值。

In [ ]:
groupby_cols = [
    "product_id",
    "product_distribution_center_id",
    "product_category",
    "product_department",
    "arrival_month",
    "product_retail_price",
    "cost",
    "discount_perc",
]
value_cols = ["shelf_days"]


df_prod = df[groupby_cols + value_cols].groupby(by=groupby_cols).mean().reset_index()

检查汇总的产品级数据。

In [ ]:
df_prod.head()

在数据中查找空值。

In [ ]:
df_prod.isna().sum() / df.shape[0]

只有`shelf_days`字段具有与没有售出商品的`product_id`对应的空值。

### 绘制数据分布图

通过生成一个箱线图来绘制聚合后的`shelf_days`字段的分布情况。

In [ ]:
df_prod["shelf_days"].plot(kind="box")

在这里，您可以看到大多数产品在抵达库存/商店后的60天内就被售出。在本教程中，您将训练一个可以预测产品在60天内售出概率的机器学习模型。

### 对分类字段进行编码

对`shelf_days`字段进行编码，生成目标字段`sold_in_2mnt`，表示产品是否在60天内售出。

In [ ]:
df_prod["sold_in_2mnt"] = df_prod["shelf_days"].apply(
    lambda x: 1 if x >= 0 and x < 60 else 0
)
df_prod["sold_in_2mnt"].value_counts(normalize=True)

将特征分隔到变量中用于模型构建。

In [ ]:
target = "sold_in_2mnt"
categ_cols = [
    "product_category",
    "product_department",
    "product_distribution_center_id",
    "arrival_month",
]
num_cols = ["product_retail_price", "cost", "discount_perc"]

对`product_department`字段进行编码。

In [ ]:
df["product_deprtment"] = (
    df["product_department"].apply(lambda x: 1 if x == "Women" else 0).value_counts()
)

对于模型构建，编码剩余的分类字段。

In [ ]:
# Create dummy variables for each categ. variable
for i in categ_cols:
    ml = pd.get_dummies(df_prod[i], prefix=i + "_", drop_first=True)
    df_new = pd.concat([df_prod, ml], axis=1)

df_new.drop(columns=categ_cols, inplace=True)
df_new.shape

将数值字段`product_retail_price`和`cost`标准化为0-1范围，使用最小-最大标准化技术。

In [ ]:
scaler = MinMaxScaler()
scaler = scaler.fit(df_new[["product_retail_price", "cost"]])
df_new[["product_retail_price_norm", "cost_norm"]] = scaler.transform(
    df_new[["product_retail_price", "cost"]]
)

训练模型

从数据框中收集所需字段。

In [ ]:
cols = [
    "discount_perc",
    "arrival_month__2",
    "arrival_month__3",
    "arrival_month__4",
    "arrival_month__5",
    "arrival_month__6",
    "arrival_month__7",
    "arrival_month__8",
    "arrival_month__9",
    "arrival_month__10",
    "arrival_month__11",
    "arrival_month__12",
    "product_retail_price_norm",
    "cost_norm",
]

将数据分成训练（80%）和测试（20%）集。

In [ ]:
X = df_new[cols].copy()
y = df_new[target].copy()
train_X, test_X, train_y, test_y = train_test_split(
    X, y, train_size=0.8, test_size=0.2, random_state=7
)

创建一个[随机森林分类器](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier)对象，并将其拟合到训练数据上。

In [ ]:
model = RandomForestClassifier(random_state=7, n_estimators=100)
model.fit(train_X[cols], train_y)

## 评估模型

在测试集上进行预测，并检查模型的准确率。

In [ ]:
pred_y = model.predict(test_X[cols])

# Calculate the accuracy as our performance metric
accuracy = metrics.accuracy_score(test_y, pred_y)
print("Accuracy: ", accuracy)

在测试集上生成混淆矩阵。

In [ ]:
confusion = metrics.confusion_matrix(test_y, pred_y)
print(f"Confusion matrix:\n{confusion}")

print("\nNormalized confusion matrix:")
for row in confusion:
    print(row / row.sum())

模型的性能可以用特异性（真阴性率）和敏感性（真阳性率）来说明。在归一化混淆矩阵中，左上角的值代表真阴性率，右下角的值代表真阳性率。

接下来，将模型保存到创建的云存储桶中以便部署。

In [ ]:
# save the trained model to a local file "model.pkl"
FILE_NAME = "model.pkl"
with open(FILE_NAME, "wb") as file:
    pickle.dump(model, file)

# Upload the saved model file to Cloud Storage
BLOB_PATH = "inventory_prediction/"
BLOB_NAME = os.path.join(BLOB_PATH, FILE_NAME)

bucket = storage.Client().bucket(BUCKET_URI[5:])

blob = bucket.blob(BLOB_NAME)
blob.upload_from_filename(FILE_NAME)

将模型上传到Vertex AI

指定以下参数以在Vertex AI Model Registry中创建模型：

- `display_name`：模型的显示名称。
- `artifact_uri`：包含模型工件及其支持文件的目录路径。
- `serving_container_image_uri`：模型服务容器的URI。

了解更多关于[Vertex AI Model Registry](https://cloud.google.com/vertex-ai/docs/model-registry/introduction)的信息。

In [ ]:
MODEL_DISPLAY_NAME = "inventory-pred-model-unique"  # @param {type:"string"}
ARTIFACT_GCS_PATH = f"{BUCKET_URI}/{BLOB_PATH}"

建立一个顶点 AI 模型资源。

确保 Sklearn 用于服务容器的版本与用于训练模型的本地版本匹配。详细了解可用的[用于顶点 AI 的预构建容器](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)。

In [ ]:
model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=ARTIFACT_GCS_PATH,
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-2:latest",
)

model.wait()

print("Display name:\n", model.display_name)
print("Resource name:\n", model.resource_name)

## 创建 Vertex AI 终端节点

为终端节点设置显示名称。

In [ ]:
ENDPOINT_DISPLAY_NAME = "inventory-pred-endpoint-unique"  # @param {type:"string"}

在Vertex AI上创建一个端点资源。

In [ ]:
endpoint = aiplatform.Endpoint.create(display_name=ENDPOINT_DISPLAY_NAME)

print("Display name:\n", endpoint.display_name)
print("Resource name:\n", endpoint.resource_name)

部署模型到创建的端点

指定用于提供部署模型所需的机器类型。

In [ ]:
MACHINE_TYPE = "n1-standard-2"

将模型部署到创建的端点。

In [ ]:
model.deploy(endpoint=endpoint, machine_type=MACHINE_TYPE)

model.wait()

print("Model display-name:\n", model.display_name)
print("Model resource-name:\n", model.resource_name)

列出部署到端点的模型，并确保库存预测模型被列出。

In [ ]:
endpoint.list_models()

## What-If 工具

What-If 工具可用于分析测试数据上的模型预测。在本教程中，What-If 工具已在前几步部署在 Vertex AI Endpoints 上的模型上进行配置和运行。

WitConfigBuilder 提供了 set_ai_platform_model() 方法，以使用在 Ai 平台模型上部署的版本配置 What-If 工具。该功能目前仅支持 Ai 平台，而不支持 Vertex AI 模型。幸运的是，还有一个选项，可以通过 set_custom_predict_fn() 方法传递一个自定义函数来生成预测，该函数可以传递本地训练的模型或返回从 Vertex AI 模型中生成预测的函数。

了解更多关于[What-If 工具](https://pair-code.github.io/what-if-tool/get-started/)。

### 准备测试样本

保存一些来自测试数据的样本，以便分析使用 What-If 工具的模型的两个可用类别（欺诈/非欺诈）。

In [ ]:
# collect some samples for each class-label from the test data
sample_size = 200
pos_samples = test_y[test_y == 1].sample(sample_size).index
neg_samples = test_y[test_y == 0].sample(sample_size).index
test_samples_y = pd.concat([test_y.loc[pos_samples], test_y.loc[neg_samples]])
test_samples_X = test_X.loc[test_samples_y.index].copy()

运行已部署的Vertex AI模型上的What-If工具

定义一个函数来从已部署的模型中获取预测，并在创建的测试数据上运行它，配置What-If工具。

In [ ]:
# configure the target and class-labels
TARGET_FEATURE = target
LABEL_VOCAB = ["not-sold", "sold"]

# function to return predictions from the deployed Model


def endpoint_predict_sample(instances: list):
    prediction = endpoint.predict(instances=instances)
    preds = [[1 - i, i] for i in prediction.predictions]
    return preds


# Combine the features and labels into one array for the What-If Tool
test_examples = np.hstack(
    (test_samples_X.to_numpy(), test_samples_y.to_numpy().reshape(-1, 1))
)

# Configure the WIT with the prediction function
config_builder = (
    WitConfigBuilder(test_examples.tolist(), test_samples_X.columns.tolist() + [target])
    .set_custom_predict_fn(endpoint_predict_sample)
    .set_target_feature(TARGET_FEATURE)
    .set_label_vocab(LABEL_VOCAB)
)

# run the WIT-widget
WitWidget(config_builder, height=800)

### 了解假设工具

在**数据点编辑器**选项卡中，您可以突出显示结果集中的一个点，并要求假设工具选择“最接近的对照事实”。这是与您选择的数据行最接近但结果相反的一行数据。左侧表中的特征是可编辑的，并可以显示需要进行哪些调整才能使特定数据行从一个结果翻转到另一个结果。例如，更改*discount_percentage*特征会显示它如何影响预测。

在**性能和公平性**选项卡下，您可以按第二个变量对预测结果进行分段。这可以让您更深入地了解数据的不同部分对模型预测的反应。例如，在以下图片中，*discount_percentage*越高，假负例越少，而*discount_percentage*越低，假正例越多。

最后，**特征**选项卡为您提供了一种直观且交互式的方式来了解数据中存在的特征。与在本笔记本中执行的探索性数据分析步骤类似，假设工具提供了关于特征的视觉和统计描述。

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- Vertex AI终端
- Vertex AI模型
- Cloud Storage存储桶（设置`delete_bucket`为*True*以进行删除）

In [ ]:
# Undeploy the model
endpoint.undeploy_all()

# Delete the endpoint
endpoint.delete()

# Delete the model
model.delete()

# Set this to true only if you'd like to delete your bucket
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI